In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

In [4]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

OVBIOUS SOLUTION USING SOFTMAX LAYER AS OUTPUT LAYER

In [6]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 955us/step - loss: 1.1516
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 848us/step - loss: 0.3858
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 835us/step - loss: 0.1797
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 873us/step - loss: 0.0990
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 862us/step - loss: 0.0662
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 831us/step - loss: 0.0641
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 784us/step - loss: 0.0553
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 0.0505
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step - loss: 0.0474
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 885us/step - loss: 0.0397


In [8]:
#to get an idea of how varied the probabilities resulting are
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[[1.9410900e-03 2.2246761e-03 9.6756148e-01 2.8272763e-02]
 [9.9542207e-01 4.3409448e-03 2.3452919e-04 2.4791664e-06]]
largest value 0.9999995 smallest value 3.0937167e-10


PREFERRED SOLUTION USING TRUE_LOGITS AND LINEAR LAYER AS OUTPUT LAYER

ALTHOUGH BY USING THIS SOLUTION WE GET THE OUTPUT AS A LIST OF NUMBERS AND NOT PROBABILITIES HENCE WE NEED TO SOFTMAX THE OUTPUT LAYER AFTERWARDS 

In [10]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note  #this is a def loss function for categorical classification
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - loss: 1.2812
Epoch 2/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - loss: 0.5836
Epoch 3/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 869us/step - loss: 0.3427
Epoch 4/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 770us/step - loss: 0.1755
Epoch 5/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 894us/step - loss: 0.1057
Epoch 6/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 735us/step - loss: 0.0702
Epoch 7/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 916us/step - loss: 0.0655
Epoch 8/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 759us/step - loss: 0.0536
Epoch 9/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 881us/step - loss: 0.0479
Epoch 10/10
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.0491


In [12]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
two example output vectors:
 [[-2.57215    -2.2937078   3.3480768   0.19853626]
 [ 6.075595    0.95841825 -1.8568461  -2.6136067 ]]
largest value 11.566255 smallest value -8.596594


In [14]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[2.5589394e-03 3.3805389e-03 9.5319533e-01 4.0865216e-02]
 [9.9352199e-01 5.9540980e-03 3.5658435e-04 1.6730362e-04]]
largest value 0.9999987 smallest value 3.908665e-08


In [16]:
for i in range(5):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-2.57215    -2.2937078   3.3480768   0.19853626], category: 2
[ 6.075595    0.95841825 -1.8568461  -2.6136067 ], category: 0
[ 4.4569287  1.1852719 -1.5025232 -2.2373922], category: 0
[-1.4202791  4.1571865 -1.5252144 -1.6121945], category: 1
[-0.41876596 -2.7845829   5.1175895  -1.3688507 ], category: 2
